## 매물 리스트 크롤링

In [ ]:
import requests
import time
import random
import pandas as pd
from tqdm import tqdm

def crawl_item_ids(property_type, geohashs):

    if property_type == "villa":
        url_format = "https://apis.zigbang.com/v2/items/villa?geohash={}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&salesTypes[2]=매매&salesPriceMin=0&domain=zigbang&checkAnyItemWithoutFilter=true"
    elif property_type == "apt":
        pass

    property_list = []
    num_geohashs = len(geohashs)
    for i, geohash in enumerate(geohashs):
        api_url = url_format.format(geohash)

        response = requests.get(api_url)

        if response.status_code == 200:
            data = response.json()
            property_list.append(data)
            print(f"[{i+1:04}/{num_geohashs:04}] {geohash} 구역 데이터 수신 완료")
        else:
            # 성공 혹은 실패 시 재수행을 위해 성공 실패 지역 이름을 기록하는 csv와 코드 필요
            # 여기에도 재실행 및 예외처리 적용
            print(f"[{i+1:04}/{num_geohashs:04}] {geohash} 구역 데이터 수신 실패 / 상태 코드: {response.status_code}")

        delay_time = random.randint(2, 10)
        time.sleep(delay_time)

    return property_list

In [17]:
property_type = "villa"
geohashs = [
    "wydmf",
    "wydmg",
    "wydmd",
    "wydme",
]
property_list = crawl_item_ids(property_type, geohashs)

[0000/0004]wydmf 구역 데이터 수신 완료
데이터 수집 완료!
[0001/0004]wydmg 구역 데이터 수신 완료
데이터 수집 완료!
[0002/0004]wydmd 구역 데이터 수신 완료
데이터 수집 완료!
[0003/0004]wydme 구역 데이터 수신 완료
데이터 수집 완료!


## 매물 리스트 저장

In [ ]:
import os

csv_dir = "./property_list"
for geohash, properties in zip(geohashs, property_list):
    csv_path = os.path.join(csv_dir, f"{property_type}_{geohash}.csv")
    df = pd.DataFrame(
        columns=["item_id", "status"],
        data=[]
    )

    datas = []
    for property in properties["items"]:
        datas.append(property["itemId"])

    df["item_id"] = datas
    df.to_csv(csv_path, index=False)

## 매물 상세 정보 저장

In [ ]:
import requests
import time
import random
from tqdm import tqdm
import json

'''
1. property_list에 있는 csv 하나를 불러온다.
2. 데이터 크롤링을 수행한다.
3. 성공하면 스테이터스에 Success, 실패하면 Fail를 기록한다.
4. property_details에 데이터를 저장한다.

'''
def crawl_property_datail(property_type, property_list_path, save_dir):
    if property_type == "villa":
        url_format = "https://apis.zigbang.com/v3/items/{}?version=&domain=zigbang"
        
    elif property_type == "apt":
        pass

    # 셔플 기능 추가하기
    df = pd.read_csv(property_list_path)
    item_ids = df[df["status"] != "success"]["item_id"].values # 성공이 아닌 데이터에 대해 크롤링 진행

    try:
        num_item_ids = len(item_ids)
        for i, item_id in enumerate(item_ids):
            api_url = url_format.format(item_id)
            print(api_url)

            response = requests.get(api_url)

            if response.status_code == 200:
                data = response.json()
                save_property_datail(data, property_type, item_id, save_dir)
                update_status("success", item_id, property_list_path)
                print(f"[{i+1:04}/{num_item_ids:04}] {item_id} 매물 데이터 수신 완료")
            else:
                update_status("fail", item_id, property_list_path)
                print(f"[{i+1:04}/{num_item_ids:04}] {item_id} 매물 데이터 수신 실패 / 상태 코드: {response.status_code}")

            delay_time = random.randint(2, 10)
            time.sleep(delay_time)
            
    except Exception as e:
        update_status("error", item_id, property_list_path)
        print(f"[{i+1:04}/{num_item_ids:04}] {item_id} 요청 중 에러 발생 → {e}")

def update_status(status, item_id, property_list_path):
    df = pd.read_csv(property_list_path)
    df.loc[df["item_id"] == item_id, "status"] = status # i가 아니라 아이디 조회 후 저장
    df.to_csv(property_list_path, index=False)

def save_property_datail(data, property_type, item_id, save_dir):
    txt_path = os.path.join(save_dir, f"{property_type}_{item_id}" + ".txt")
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(data, indent=2, ensure_ascii=False))

In [36]:
from glob import glob

property_list_paths = glob("./property_list/*.csv")
property_details_save_path = "./property_details"
property_type = "villa"

for property_list_path in property_list_paths:
    crawl_property_datail(property_type, property_list_path, property_details_save_path)

https://apis.zigbang.com/v3/items/44280486?version=&domain=zigbang
[0001/0019] 44280486 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/44111614?version=&domain=zigbang
[0002/0019] 44111614 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/44162625?version=&domain=zigbang
[0003/0019] 44162625 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/44251738?version=&domain=zigbang
[0004/0019] 44251738 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/44253957?version=&domain=zigbang
[0005/0019] 44253957 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/44304825?version=&domain=zigbang
[0006/0019] 44304825 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/44254039?version=&domain=zigbang
[0007/0019] 44254039 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/44299071?version=&domain=zigbang
[0008/0019] 44299071 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/43789259?version=&domain=zigbang
[0009/0019] 43789259 매물 데이터 수신 완료
https://apis.zigbang.com/v3/items/44290735?version=&domain=zigbang
[0010/0019] 44290735 매물 